# 신경망 모델 구성하기
<hr>

신경망은 데이터에 대한 연산을 수행하는 layer/module로 구성되어 있다.

torch.nn 네임스페이스는 신경망을 구성하는데 필요한 모든 구성 요소를 제공.

PyTorch의 모든 모듈은 nn.Module의 subclass이다.

신경망은 다른 모듈(layer)로 구성된 모듈이다.
중첩된 구조를 통해 복잡한 아키텍처를 쉽게 구축하고 관리할 수 있다.

FashionMNIST 데이터셋의 이미지들을 분류하는 신경망 구성

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# 학습을 위한 장치 얻기
<hr/>


GPU를 사용하는 것이 가능한지 불가능한지 확인 후 가능하면 사용, 불가능하면 cpu 사용

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')

Using cpu device


# Class 정의하기
<hr>

신경망 모델을 nn.Module 의 subclass로 정의하고, &#95;&#95;init&#95;&#95; 에서 신경망 계층들을 초기화한다.

nn.Module 를 상속받은 모든 클래스는 forward method 에 입력 데이터에 대한 연산들을 구현한다.

In [5]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


NeuralNetwork의 instance를 생성하고 이를 device로 이동한 뒤, structure를 출력.

In [6]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


모델을 사용하기 위해 입력 데이터 전달.
이는 일부 백그라운드 연산들과 함께 모델의 forward를 실행.

주의!!!!
model.forward()를 직접 호출 x
-> 이유 : model(입력데이터) 이렇게 하면 자동으로 forward를 진행

모델에 입력을 호출 시, 각 class에 대한 raw prediction value가 있는 10-차원 tensor가 반환된다.

이 값을 nn.Softmax 모듈의 인스턴스에 통과시켜 확률값을 얻는다.

In [21]:
X = torch.rand(1, 28, 28, device = device)
logits = model(X)

pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)

print(f'Predicted class : {y_pred}')

Predicted class : tensor([0])


# 모델 계층
<hr>

FashionMNIST 모델의 계층 살펴보기

이를 설명하기 위해
28*28 크기의 이미지 3개로 구성된 미니배치를 가져와 신경망을 통과할 때 발생하는 일을 알아보겠다.


In [24]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


# nn.Flatten()
<hr>

nn.Flatten 계층을 초기화하여 28*28의 2D 이미지를 784 픽셀 값을 갖는 연속된 배열로 변환
(dim=0의 미니배치 차원은 유지된다.)

In [25]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


# nn.Linear()
<hr>

선형 계층은 저장된 weight와 bias를 이용하여 입력에 linear transformation을 적용하는 모듈이다.

In [26]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


# nn.ReLU

nonlinear activation은 모델의 입력과 출력 사이의 복잡한 mapping을 만든다.  

비선형 활성화는 선형 변환 후에 적용되어 nonlinearity를 도입하고, 신경망이 다양한 현상을 학습할 수 있게 돕는다.


ReLU = max(0,x)
  
음의 영역은 0으로,  
양의 영역은 그대로 출력

In [27]:
print(f'Before ReLU: {hidden1}\n\n')
hidden1 = nn.ReLU()(hidden1)
print(f'After ReLU: {hidden1}')

Before ReLU: tensor([[ 0.2139,  0.0729, -0.1816,  0.2750, -0.0096, -0.2355, -0.1877,  0.5390,
         -0.0523,  0.2740, -0.0568, -0.0949,  0.1704,  0.1565,  0.0331, -0.2852,
          0.6673,  0.0736, -0.9740,  0.0222],
        [-0.1393,  0.3397, -0.0581,  0.2144, -0.1123, -0.3525, -0.1446,  0.4469,
          0.0166, -0.0500, -0.0148, -0.0832,  0.0941,  0.1262,  0.1334, -0.2588,
          0.6414,  0.3607, -0.8543, -0.1357],
        [ 0.0431,  0.5149, -0.0912,  0.4047, -0.1594,  0.0242, -0.1049,  0.3555,
         -0.1224,  0.1461, -0.0793, -0.0594,  0.3898,  0.0957,  0.2798, -0.2219,
          0.6037, -0.0439, -0.6913,  0.1195]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.2139, 0.0729, 0.0000, 0.2750, 0.0000, 0.0000, 0.0000, 0.5390, 0.0000,
         0.2740, 0.0000, 0.0000, 0.1704, 0.1565, 0.0331, 0.0000, 0.6673, 0.0736,
         0.0000, 0.0222],
        [0.0000, 0.3397, 0.0000, 0.2144, 0.0000, 0.0000, 0.0000, 0.4469, 0.0166,
         0.0000, 0.0000, 0.0000, 0.0941, 0.1262, 0.13

# nn.Sequential

nn.Sequential은 순서를 갖는 모듈의 컨테이너이다.  
데이터는 정의된 것과 같은 순서로 모든 모듈들을 통해 전달된다.  
sequential container를 사용하여 아래의 seq_modules와 같은 신경망을 빠르게 만들 수 있다.

In [28]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20,10)
)

input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

# nn.Softmax

신경망의 마지막 선형 계층은 nn.Softmax 모듈에 전달될 logits을 반환한다.  

logits은 모델의 각 class에 대한 예측 확률을 나타내도록 [0,1] 값으로 scaling 된다. 

dim=1 은 값의 합이 1이되는 차원을 나타내는 것.

In [29]:
softmax = nn.Softmax(dim = 1)
pred_probab = softmax(logits)

# 모델 매개변수

신경망 내부의 많은 계층들은 매개변수화 된다.  

즉 학습 중에 최적화되는 weight와 bias와 연관되어진다. nn.Module를 상속하면 모델 객체 내부의 모든 필드들이 자동으로 추적되며, 모델의 parameters() 및 named_parameters() 메소드로 모든 매개변수에 접근이 가능해진다.

이 예제에서는 각 매개변수들을 iterate 하며 매개변수의 크기와 값을 출력한다.

In [33]:
print(f'Model structure: {model}\n\n')

for name, param in model.named_parameters():
    print(f'Layer: {name} | Size: {param.size()} | Values : {param[:2]}\n')


Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0065, -0.0033, -0.0307,  ..., -0.0048, -0.0166, -0.0302],
        [ 0.0154,  0.0276,  0.0313,  ...,  0.0165, -0.0342, -0.0237]],
       grad_fn=<SliceBackward0>)

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0141, -0.0252], grad_fn=<SliceBackward0>)

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0219,  0.0262, -0.0309,  ...,  0.0009,  0.0033,  0.0179],
        [ 0.0407,  0.0011,  0.0258,  ...,  0.0395, -0.0352, -0.0438]],
       grad_fn=<SliceBackward0>)

Layer: linear_relu_stack.2.bias | Siz